# <font color='green'>ĐỒ ÁN 1: THU THẬP DỮ LIỆU
## Danh sách các thành viên và bảng phân công công việc:
| STT | Họ và tên | MSSV | Nội dung công việc | Hoàn thành |
| :--- | :---: | :---: | :---: | ---: |
| 01 | Lưu Trường Dương | 19120489 |Đọc và xử lý dữ liệu playlists phần parse HTML, xử lý bỏ qua lỗi của web driver và click accept | 100%|
| 02 | Nguyễn Phạm Quang Dũng | 19120485 | Đọc và xử lý dữ liệu tracks phần parse HTML, xử lý link của các track khi thu được, tối ưu thời gian chờ load web | 100% |
| 03 | Huỳnh Quốc Duy | 19120494 | Đọc và xử lý dữ liệu users phần parse HTML, chỉnh sửa lại các file sau khi đã Crawl xuống | 100% |
| 04 | Phạm Đức Huy | 19120534 | Đọc và xử lý dữ liệu playlists & tracks phần API, xử lý trackIds | 100% |
| 05 | Lê Thành Lộc | 19120562 | Đọc và xử lý dữ liệu users phần API & làm markdown, tự động tìm client_id | 100% |

## Import các package cần thiết

In [ ]:
import requests
import requests_cache
from bs4 import BeautifulSoup as bs
import time
import json
import pandas as pd
from datetime import datetime,timedelta
from selenium import webdriver
from requests_html import HTML
import lxml
import urllib.robotparser
import csv
#wait
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


## Kiểm tra xem website có cho thu thập dữ liệu không? 

In [ ]:
#check robots.txt
def check_robots(url):    
    rp = urllib.robotparser.RobotFileParser()
    rp.set_url(url)
    rp.read()
    check  = rp.can_fetch("*",url)
    if not check:
        print("Khong the get url tu trang web nay")
        exit()

## Setup các biến toàn cục

In [ ]:
#luu link cac playlist
link_playlist = []
#dung de luu link cac track
link_alltrack = []
# dung de luu link cac user
link_alluser =[] 


## Quá trình chạy Web Driver
- Dùng webdriver của module selenium để chạy những trang web yêu cầu phải chạy javascript mới cho lấy dữ liệu từ biến
#### Quá trình chạy webdriver trải qua các bước sau:
   - Kiểm tra xem trang web có cho phép thu thập dữ liệu hay không với `robots.txt`
   - Bỏ qua thông báo các lỗi `ssl-error-code-1` (các lỗi trong khung hồng phía dưới)
   - Dùng WebDriverWait của selenium để đợi một thời gian ngắn ( ở đây là 20 giây ) để trang web load xong (khi tìm thấy id = 'onetrust-accept-btn-handler') , nếu quá thời gian chờ thì sẽ thông báo bị TimeOut và thoát webdriver
   - Click button `"Accept all cookies"` để có thể bắt đầu thu thập dữ liệu (để có thể lấy giá trị các biến từ tag script của trang web) bằng cách tìm ô chứa id = onetrust-accept-btn-handler và click vào đó
   
#### Các biến quan trọng:

   - `url`: link đi tới trang web [SoundCloud](https://soundcloud.com/)
   - `key_word`: danh sách các thể loại nhạc cho biết muốn tìm các playlist với các từ khóa nào

In [ ]:
#chay driverWeb
url = "https://soundcloud.com"

key_word = ['indie','pop','country','lofi','rock','rap','edm','baroque','piano','jazz','electric','chill']
# check_robots(url+'/search/sets?q=indie')

#bo qua thong bao cac loi ssl-error-code-1
options = webdriver.ChromeOptions()
options.add_argument("--ignore-certificate-error")
options.add_argument("--ignore-ssl-errors")
driver = webdriver.Chrome(executable_path='chromedriver.exe')
driver.get(url)

#doi trang load xong bang cach tim id = "onetrust-accept-btn-handler"
try:
    main =  WebDriverWait(driver,20).until(
        EC.presence_of_element_located((By.XPATH,'//*[@id="onetrust-accept-btn-handler"]'))
    )
except:
    print('TimeOut')
    driver.quit()
#click accept
accept = driver.find_element_by_id('onetrust-accept-btn-handler').click()


## Thu thập danh sách các link playlist 
#### Quá trình thu thập danh sách link playlist
   - Duyệt qua từng thể loại nhạc trong `key_word` để bắt đầu quá trình thu thập.
   - Vì muốn tìm dữ liệu bắt đầu từ playlist nên ta sẽ duyệt thằng vào đường link tải các playlist có tên từ khoá trong key_word. Đường link sẽ là một string được ghép bởi biến toàn cục `url` và f-string `f'/search/sets?q={key_word[i]}'` 
   - Dùng WebDriverWait để đợi load xong trang web (dấu hiệu nhận biết là tìm thấy class tên là `searchList__item`. Đây là class xuất hiện trong `searchList`, cửa sổ thường được load sau cùng)
   - Để tăng tối đa tốc độ cuộn của trang web thì ở đây ta sẽ dùng vòng lặp while chạy liên tục và tính thời gian bằng time.time(). Ở đây ta sẽ scroll 15 lần và mỗi lần scroll di chuyển một đoạn `document.body.scrollHeight`. Trong quá trình thì ta sẽ cộng biến run_time với khoảng thời gian từ khi bắt đầu vòng while cho tời tời điểm so sánh, cứ như vậy nếu scroll thì sẽ trả về lại run_time = 0 còn không thì sẽ diễn ra TimeOut khi run_time > max_run_time. Ngoài ra với mỗi lần scroll thành công thì biến `flag` sẽ tự cộng thêm 1 đơn vị.
   - Dừng `5(s)` để trang web load xong và thu thập toàn bộ các link playlist mà web vừa load được sau quá trình scroll
   - Dùng CSS Selector để tìm các link playlist từ trang web mà ta vừa load qua attrs `href`
   - extend link vừa thu được vào list toàn cục link_playlist. Dùng pandas.unique để lọc những link bị trùng với nhau. Nếu số lượng link >1000 thì dừng quá trình get lại.

In [ ]:
#dung de lay link_playlist
for i in range(0,len(key_word)):
    driver.get(url+ f'/search/sets?q={key_word[i]}')
    #xu ly thoi gian doi webload
    try:
        main =  WebDriverWait(driver,10).until(
            EC.presence_of_element_located((By.CLASS_NAME,"searchList__item"))
        )
    except:
        continue
   
    #scroll
    pre_scroll_height = driver.execute_script('return document.body.scrollHeight;')
    run_time= 0
    max_run_time =10
    flag = 0

    while flag<15:
        iteration_start = time.time()
     
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')

        post_scroll_height = driver.execute_script('return document.body.scrollHeight;')

        scrolled = post_scroll_height != pre_scroll_height
        timed_out = run_time >= max_run_time

        if scrolled:
            run_time = 0
            pre_scroll_height = post_scroll_height
            flag +=1
        elif not scrolled and not timed_out:
            run_time += time.time() - iteration_start
        elif not scrolled and timed_out:
            break  
    #dung 5 giay de load het trang
    time.sleep(5)
    soup  = bs(driver.page_source, 'lxml')
    temp  =  soup.select('div.soundTitle__usernameTitleContainer a.sc-link-primary.soundTitle__title.sc-link-dark.sc-text-h4')
    list_search = [link['href'] for link in temp]
    link_playlist.extend(list_search)
    link_playlist = pd.unique(link_playlist).tolist()
    if len(link_playlist) > 1000:
        break


### Lưu danh sách các link playlist vừa thu thập được vào file `link_playlist.txt`

In [ ]:
#save all link_playlist get to link_playlist.txt
writer = open('link_playlist.txt','wt',encoding="utf-8")
for i in link_playlist:
    writer.write(i)
    writer.write('\n')
writer.close()
    

### Đọc danh sách các link playlist từ `link_playlist.txt`

In [ ]:
#read link_playlist from link_playlist.txt
link_playlist.clear()
reader = open('link_playlist.txt','r')
for i in reader:
    link_playlist.append(i.strip('\n'))
reader.close()

### tạo một hàm `convert_track_link`
- Để xử lý link track trong quá trình get. Vì `link trang khi ghép được`  = `link track gốc` + `thông tin playlist chứa nó`. Mục đích để có thể sử dụng sau này nếu như playlist không còn chứa track này nữa, và để xử lý việc trùng lặp link (do 1 track có thể nằm tại nhiều playlist khác nhau nên 1 track có thể có nhiều link) cũng như để link được gọn và đẹp hơn khi thêm vào file csv

In [ ]:
#Xu ly link_alltrack track thu duoc tu playlist de cho duong link dep hon
def convert_track_link(track_link):
    temp = track_link.split('/')
    temp_1 = temp[2].split('?')[0]
    result = f'/{temp[1]}/{temp_1}'
    return result

## Extract các links playlist bằng parse HTML và lưu thông tin vào `playlist_temp.csv`
### Lưu ý:
   - Đặt tên là `playlist_temp.csv` vì cần phải xử lý mới ra được file playlist.csv đúng với yêu cầu đề bài
   - Dù đã lưu các link vào file nhưng có thể trong quá trình get link, do user thay đổi đường link cá nhân của họ dẫn đến link bị sai do chưa cập nhật lại
   - Do có một số track không cho phát tại Việt Nam nên trong quá trình, có thể số track > 4 nhưng chỉ lấy về 0 hoặc 1 track của playlist
### File `playlist_temp.csv` gồm các cột:
   - `ID`: số thứ tự của playplist nhưng vì không get được id nên dùng làm id luôn
   - `Name`: tên của playlist 
   - `Link`: đường link tới playlist
   - `UserName`: tác giả của playlist
   - `Type`: Thể loại playlist, ở trên web được gọi là tag, trả về None nếu không có
   - `NumberTrack`: số track trong playlist
   - `Duration`: tổng thời gian cần để phát hết playlist 
   - `DateCreate`: thời gian tạo playlist
   - `Like`: số lượt like của playlist
   - `Repost`: số lượng đăng lại của playlist
   - `TrackIds`: là một string chứa danh sách id của các track trong playlist (Hiện tại chỉ chứa string là các link track, sau khi xử lý ở giai đoạn sau crawl mới thành id)
   
### Xử lý:
   - Mỗi lần get thì dùng `WebDriverWait` để đợi, nếu TimeOut thì sẽ chuyển qua get link tiếp theo.
   - Những thông tin riêng lẽ thì có thể select riêng.
   - Dùng `trackSummary` để có thể select cùng lúc `playlist_numbertrack` và `playlist_duration`
   - Dùng `playlist_miniStats` để select cùng lúc `like` `repost`
   - Thu thập danh sách các `link_user` và lưu vào `link_alluser` sau đó lọc các users bị trùng lặp
   - Thu thập danh sách các `link_track` và lưu vào `link_alltrack` sau đó lọc các tracks bị trùng lặp
       - Chuyển `list_track` thành string rồi lưu nó vào trackIds
   - Cả `users` và `tracks` đều chỉ lấy 4 phần tử đầu do sợ số lượng quá nhiều, thời gian crawl lâu.
   - Do có một số playlist để ẩn track ( private) nên trong quá trình get vẫn lấy được `duration`, nên những trường hợp này trả `duration` về 0 luôn
### Phần code được comment lại ở chỗ xử lý playlist
   - Nếu tốc độ mạng được đảm bảo ổn định thì có thể dùng đoạn code comment để có thể trực tiếp tìm ra các list ID và trực tiếp lưu vào `trackIds` luôn mà không cần phải xử lý dữ liệu sau khi Crawl về nữa. Bằng cách tìm index phần tử của list vừa thu thập được trong `list_alltrack` ( do `list_alltrack` được cập nhật từ các `link_track`)
   - Nhưng để phòng trường hợp quá trình get bị lỗi thì nên để sau khi crawl về rồi xử lý data sau.

In [ ]:

#lay info playlist va luu vao playlist.csv
with open('./Crawl_data/playlist_temp.csv','w',encoding="utf-8",newline="") as f:
    writer = csv.writer(f)     
    playlist_column = ['ID','Name','Link','UserName','Type','NumberTrack','Duration','DateCreate','Like','Repost','TrackIds']
    writer.writerow(playlist_column)
    for index in range(0,len(link_playlist)): 
        driver.get(url+link_playlist[index])   
        try:
            main =  WebDriverWait(driver,10).until(
                EC.presence_of_element_located((By.CLASS_NAME,"listenNetworkSidebar"))
            )
        except:
            print('TimeOut')
            #neu bi time out thi bo qua link nay
            continue
        soup  = bs(driver.page_source, 'lxml')
        #playlist_id
        playlist_id = index

        #playlist_name
        playlist_name = soup.select('div.soundTitle__usernameTitleContainer h1 span')[0].get_text() \
            if soup.select('div.soundTitle__usernameTitleContainer h1 span') \
                else 'None'
        #playlist_link
        playlist_link=link_playlist[index]
        #playlist_user
        playlist_username = soup.select('div.soundTitle__usernameHeroContainer h2 a')[0].text.strip().strip('\n') \
            if soup.select('div.soundTitle__usernameHeroContainer h2 a') \
                else 'None'
        #playlist_type
        playlist_type = soup.select('div.fullHero__foreground.fullListenHero__foreground div.fullHero__info a span')[0].text \
            if soup.select('div.fullHero__foreground.fullListenHero__foreground div.fullHero__info a span') \
                else 'None'
        #playlist_numbertrack
        playlist_numbertrack = '0'
        #playlist_duration ( tong thoi gian cua cac track)
        playlist_duration = '0'

        #tracksSummary
        tracksSummary = soup.select('div.fullHero__tracksSummary div div div') 
        if tracksSummary:   
            playlist_numbertrack = soup.select('div.fullHero__tracksSummary div div div')[0].text  
            if '0' == playlist_numbertrack:
                #Xu ly o day boi vi co mot so track ma user an di, nhung van hien tong thoi gian duration mac gi so track public la 0
                playlist_duration = '0'
            else:
                playlist_duration = soup.select('div.fullHero__tracksSummary div div div')[3].text     
        #playlist_date
        playlist_date = soup.select('div.fullHero__info div.fullHero__uploadTime.sc-type-medium.sc-text-body time')[0]['datetime'] \
            if soup.select('div.fullHero__info div.fullHero__uploadTime.sc-type-medium.sc-text-body time') \
                else '0'
            
        #playlist_ministats
        playlist_like = '0'
        playlist_repost = '0'
        playlist_ministats = soup.select('div.listenEngagement__footer ul li')
        for i in range(0,len(playlist_ministats)):
            if 'like' in playlist_ministats[i]['title']:
                playlist_like = playlist_ministats[i]['title'].split()[0]
            elif 'repost' in playlist_ministats[i]['title']:
                playlist_repost = playlist_ministats[i]['title'].split()[0]
        
        users = soup.select('div.trackItem__content.sc-truncate a.trackItem__username.sc-link-light.sc-link-secondary')
        link_user = []
        for i in range(0,len(users)):
            link_user.append(url+ users[i]['href'])
            #lay 4 user link dau tien xuat hien trong playlist neu so user >= 4
            if 3==i:
                break
        link_alluser.extend(link_user)
        #Xu ly duplicate bang pandas
        link_alluser = pd.unique(link_alluser).tolist()

        tracks = soup.select('div.trackItem__content.sc-truncate a.trackItem__trackTitle.sc-link-dark.sc-link-primary.sc-font-light')
        playlist_trackIds = []
        for i in range(0,len(tracks)):
            temp_track = convert_track_link(tracks[i]['href'])
            playlist_trackIds.append(url+ temp_track)
            #lay 4 track link dau tien xuat hien trong playlist neu so track >= 4
            if 3==i:
                break
        link_alltrack.extend(playlist_trackIds)
        #Xu ly duplicate
        link_alltrack = pd.unique(link_alltrack).tolist()
        # playlist_track = []
        #o day ghi truc tiep, nhung de phong trong qua trinh crawl ma bi loi timeout thi nen xy ly sau khi lay data xuong
        # #ghi thong tin cua cot playlist_track/tracklds ( theo nhu de bai)
        # for track in playlist_trackIds:
        #     #o day co the bi sai neu track khong nam trong link_alltrack
        #     #theo logic thi khong sai duoc
        #     playlist_track.append(link_alltrack.index(track))
        # #Xu ly 1 track co the duoc lap lai nhieu lan trong playlist
        # playlist_track = pd.unique(playlist_track).tolist()
        playlist_trackIds = ';'.join(str(v) for v in playlist_trackIds)
    
        row = [playlist_id,playlist_name,playlist_link,playlist_username,playlist_type,playlist_numbertrack,playlist_duration,playlist_date,playlist_like,playlist_repost,playlist_trackIds]
        writer.writerow(row)


### Lưu thông tin track vào file `link_alluser.txt` và user vào `link_alltrack.txt`

In [ ]:
writer1 = open('link_alluser.txt','wt',encoding="utf-8")
for i in link_alluser:
    writer1.write(i)
    writer1.write('\n')
writer1.close()
writer2 = open('link_alltrack.txt','wt',encoding="utf-8")
for i in link_alltrack:
    writer2.write(i)
    writer2.write('\n')
writer2.close()


### Clear link_alltrack và link_alluser để lấy dữ liệu từ file `link_alltrack.txt` và `link_alluser.txt` sau khi đã xử lý phía trên

In [ ]:
#clear list de dam bao khong con du lieu duoc luu trong list
link_alltrack.clear()
link_alluser.clear()
#lay thong tin
reader = open('link_alltrack.txt','r')
for i in reader:
    link_alltrack.append(i.strip('\n'))
reader.close()
reader = open('link_alluser.txt','r')
for i in reader:
    link_alluser.append(i.strip('\n'))
reader.close()


## Extract các links track bằng parse HTML và lưu thông tin vào `track_temp.csv`
### File track_temp.csv gồm các cột sau:
   - `ID`: số thứ tự của track tương tự như track do không lấy đươc id nên ta sẽ coi đây như id của user
   - `Name`: tên của track
   - `Link`: đường link tới track
   - `UserName`: tên tác giả của track
   - `Type`: loại track
   - `Time`: thời lượng của track
   - `Date`: thời gian tạo track
   - `Played`: số lượng người nghe của track 
   - `Like`: số lượt like của track
   - `Repost`: số repost của track
   - Ngoài ra có một số trang có thể lấy được số comment nhưng do cấu trúc đường dẫn html khác và số lượng rất ít nên không có làm thêm trong bài (Những link này sẽ được loại bỏ sau khi crawl data xong)
### Xử lý:
   - Do chỉ cần tìm đúng tag chưa thông tin của cột nên ta chỉ cần dùng CSS Selector như bình thường
   - Tương tự dùng WebDriverWait để đợi trang web load xong bằng cách tìm `class localeSelector`, class này nằm trong ô `language`, thường nằm ở cuối trang nên khi tìm được class này thì trang gần như load xong ( có thể có sai sót nhưng rất ít)
   - `track_ministats` là list các tag `li` có chứa thông tin về lượt nghe, lượt like và lượt đăng lại nên ta sẽ lấy các thông tin trên dựa vào list này
### Lưu ý:
   - Do hiện [SoundCloud](https://soundcloud.com/) đang có nhiều kiểu trình bày trang track nên trong quá trình get, những trang được thiết kế theo kiểu mới, khi get data sẽ bị lỗi (chiếm số lượng ít hơn nhiều so với những track get được). Nguyên nhân nằm ở `track_ministats` (đã xử lý những trường hợp này sau khi crawl data xong)
   - Cũng có thể do track này để chế độ private hoặc không cho phát ở khu vực Việt Nam nên quá trình get có thể bị lỗi, không cho ra thông tin về stats [ví dụ](https://soundcloud.com/atlantic-records-uk/grouplove-colours)

In [ ]:
#xu li link_alltrack
with open('./Crawl_data/track_temp.csv','w',encoding="utf-8",newline="") as f:
    writer = csv.writer(f)    
    track_column = ['ID','Name','Link','UserName','Type','Time','Date','Played','Like','Repost']
    writer.writerow(track_column)
    for index in range(0,len(link_alltrack)): #len(link_alltrack)
        driver.get(link_alltrack[index])
        try:
            main =  WebDriverWait(driver,10).until(
                EC.presence_of_element_located((By.CLASS_NAME,"localeSelector"))
            )
        except:
            print('TimeOut')
            continue
        #   driver.quit()        
        soup  = bs(driver.page_source, 'lxml')
        #Track_ID
        track_id = index
        #Track_name
        track_name = soup.select('div.soundTitle__usernameTitleContainer div.soundTitle__usernameHeroContainer h2 a')[0].text.strip().strip('\n') \
            if soup.select('div.soundTitle__usernameTitleContainer div.soundTitle__usernameHeroContainer h2 a') \
                else 'None'
        #track_link
        track_link = link_alltrack[index]
        #track_user
        track_user = soup.select('div.soundTitle__titleContainer div.soundTitle__usernameTitleContainer h1 span')[0].text \
            if soup.select('div.soundTitle__titleContainer div.soundTitle__usernameTitleContainer h1 span') \
                else 'None'
        #track_type
        track_type = soup.select('div.fullHero__foreground.fullListenHero__foreground div.fullHero__info a')[0].text \
            if soup.select('div.fullHero__foreground.fullListenHero__foreground div.fullHero__info a') \
                else 'None'
        #track_time
        track_time = soup.select('div.playbackTimeline__duration span')[1].text \
            if soup.select('div.playbackTimeline__duration span') \
                else '0'
        #track_date
        track_date = soup.select('div.fullHero__info div.fullHero__uploadTime.sc-type-medium.sc-text-body time')[0]['datetime'] \
            if soup.select('div.fullHero__info div.fullHero__uploadTime.sc-type-medium.sc-text-body time') \
                else '0'
        #track_ministats
        track_played = '0'
        track_like = '0'
        track_repost = '0'
        track_ministats = soup.select('div.listenEngagement__footer ul li')
        #track_played
        #track_like
        #track_repost
        for i in range(0,len(track_ministats)):
            if  'play' in track_ministats[i]['title']:
                track_played = track_ministats[i]['title'].split(' ')[0]
            if  'like' in track_ministats[i]['title']:
                track_like = track_ministats[i]['title'].split(' ')[0]
            if  'repost' in track_ministats[i]['title']:
                track_repost = track_ministats[i]['title'].split(' ')[0]
        row = [track_id,track_name,track_link,track_user,track_type,track_time,track_date,track_played,track_like,track_repost]
        writer.writerow(row)
        
        

## Extract các links user bằng parse HTML và lưu thông tin vào `user1.csv`
### File user1.csv gồm các cột sau:
   - `ID`: số thứ tự của user, do không trích được id nên ta coi như đây là id của user.
   - `DisplayName`: tên hiển thị của user
   - `Verified`: Nếu tài khoản user này đã được `verified` thì sẽ có giá trị verifed còn không thì trả về `None`
   - `Name/Country`: FullName và tên Quốc gia của user, nếu chỉ có 1 thuộc tính thì có thể hoặc đó là FullName hoặc đó là tên Quốc gia
   - `CreaterBadge`: gói soundcloud mà user sử dụng
   - `Link`: đường link tới user
   - `Follower`: số người đang theo dõi user
   - `Following`: số người mà user đang theo dõi
   - `Tracks`: số bài hát của user
   - `Facebook`: đường link tới Facebook của user
   - `Instagram`: đường link tới Instagram của user
   - `Twitter`: đường link tới Twitter của user
### Xử lý:
   - Tương tự dùng CSS Selector để lấy thông tin mỗi row
   - `user_social` để lưu các tag có thông tin về mạng xã hội cá nhân
   - Có thể lấy nhiều hơn link mạng xã hội cá nhân nhưng ở đây chỉ lấy 3 cái là Facebook, Instagram, Twitter vì được dùng phổ biến nhất
   - `addition info` chứa các thẻ chưa thông tin về `Name/Country` và `CreateBadge`. Vì `addition info` có nhiều trường hợp nên ta phải giải quyết riêng
   - Ở thông tin like, do có một tag chứa thông tin `Fans also like` giống với `like` nên ta phải xử lý thêm.
   - Do khi lấy bị dính thêm text của thẻ span con `verified` nên ta cũng cần phải xử lý lại `DisplayName`

In [ ]:
#Xu ly link_alluser
with open('./Crawl_data/user.csv','w',encoding="utf-8",newline="") as f:
    writer = csv.writer(f)    
    user_column = ['ID','DisplayName','Verified','Name/Country','CreaterBadge','Link','Follower','Following','Tracks','Like','Facebook','Instagram','Twitter']
    writer.writerow(user_column)
    for index in range(0,len(link_alluser)):
        driver.get(link_alluser[index])        
        try:
            main =  WebDriverWait(driver,10).until(
                EC.presence_of_element_located((By.CLASS_NAME,"localeSelector"))
            )
        except:
            print('TimeOut')
            # driver.quit()   
            continue
        soup  = bs(driver.page_source, 'lxml')
        #user_stt
        user_id = index
        #user_displayname
        user_displayname = soup.select('div.profileHeaderInfo.sc-media div.profileHeaderInfo__content.sc-media-content h2')[0].text.strip().strip('\n').strip('Verified').strip().strip('\n') \
            if soup.select('div.profileHeaderInfo.sc-media div.profileHeaderInfo__content.sc-media-content h2') \
                else 'None'
        #user_verified
        user_verified = soup.select('div.profileHeaderInfo.sc-media div.profileHeaderInfo__content.sc-media-content h2 div span span')[0].text \
            if soup.select('div.profileHeaderInfo.sc-media div.profileHeaderInfo__content.sc-media-content h2 div span span') \
                else 'None'
        
        #addition_info
        #user_name_or_country    
        user_name_or_country=''
        #user_badge
        user_badge=''
        user_addition = soup.select('div.profileHeaderInfo.sc-media div.profileHeaderInfo__content.sc-media-content h3')
        if len(user_addition) == 3:
            user_name_or_country = user_addition[0].string + '/' +user_addition[1].string
            user_badge = user_addition[2].text.strip().strip('\n')
        elif len(user_addition) == 2:
            user_name_or_country = user_addition[0].string
            user_badge = user_addition[1].text.strip().strip('\n')
        elif len(user_addition) == 1:
            user_name_or_country = 'None'
            user_badge = user_addition[0].text.strip().strip('\n')
        if not user_badge:
            user_badge = 'None'
        #user_link
        user_link = link_alluser[index]
        #user_follower
        user_follower ='0'
        #user_Following
        user_following = '0'
        #user_tracks
        user_tracks = '0'
        #user_stats
        user_stats = soup.select('article.infoStats table tbody tr td a')
        for i in range(0,len(user_stats)):
            if 'follower' in  user_stats[i]['title']:
                user_follower = user_stats[i]['title'].split()[0]
            elif 'Following' in user_stats[i]['title']:
                user_following = user_stats[i]['title'].split()[1]
            elif 'track' in user_stats[i]['title']:
                user_tracks = user_stats[i]['title'].split()[0]
        #user_like
        user_like ='0'
        user_like_temp = soup.select('span.sidebarHeader__actualTitle.sc-text-h3') \
            if soup.select('span.sidebarHeader__actualTitle.sc-text-h3') \
                else 'None'
        for like in user_like_temp:
            if 'like' in like.text and 'Fans' not in like.text:
                user_like = like.text.split()[0]
        #user_facebook
        user_facebook='None'
        #user_instagram
        user_instagram='None'
        #user_twitter
        user_twitter='None'
        #user_social
        user_social = soup.select('article.infoStats div.web-profiles ul li')
        for i in range(0,len(user_social)):
            if 'acebook' in user_social[i].div.a.text:
                user_facebook = user_social[i].div.a['href']
            if 'nstagram' in user_social[i].div.a.text:
                user_instagram = user_social[i].div.a['href']
            if 'witter' in user_social[i].div.a.text:
                user_twitter = user_social[i].div.a['href']
        row = [user_id,user_displayname,user_verified,user_name_or_country,user_badge,user_link,user_follower,user_following,user_tracks,user_like,user_facebook,user_instagram,user_twitter]
        writer.writerow(row)

### Mở 2 file playlist_temp và track_temp vào 2 DataFrame df1 và df2 bằng pandas

In [ ]:
df1 = pd.read_csv('./Crawl_data/playlist_temp.csv')
df2 = pd.read_csv('./Crawl_data/track_temp.csv')

### Xử lý cho gọn file track_temp.csv:
- Đầu tiên ta sẽ xoá những row có `Date = '0'`. Bằng cách tìm vị trí (index) có `Date = '0'`, sau đó dùng drop với `errors='ignore'`( phải set giá trị errors='ignore' loại bỏ hẳn các row bị drop)
- Cập nhật lại index, sau đó cập nhật lại ID/STT theo index 

In [ ]:
#loai nhung track ma khi lay xuong thong tin bi loi trong qua trinh get
list_drop_track = []
list_drop_track.clear()
for i in range(0,len(df2)):
    if df2.Date[i] == '0':
        list_drop_track.append(i)

for i in list_drop_track:
    df2 = df2.drop(i,errors='ignore')
#cap nhat lai index
df2 = df2.reset_index(drop=True)
# #chinh lai soID/STT cho dep
for j in df2.index:
    df2.at[j,'ID'] = j


### Cập nhật lại TrackIds cho df1( playlist) và lưu file:
- Bằng cách tìm ID của những link ở column `TrackIds` trong column `Link` của track
- Lưu df1 và df2 sau khi được xử lý thành file Playlist.csv và Track.csv ( thực ra có thể thay đổi trực tiếp nhưng để đề phòng rủi ro trong quá trình xử lý data, tránh phải crawl lại data thì ta tách thành 2 file riêng)

In [ ]:
#do thay doi data tu thu muc goc nen cell nay chi duoc phep chay 1 lan, chay lan thu 2 se bi loi
trackIDs = []
link_track = df2['Link'].tolist()
for i in range(0,len(df1['TrackIds'])):
    if len(str(df1['TrackIds'][i])) == 0 :
        df1.at[i,'TrackIds'] = ''
    else:
        temp4 = str(df1['TrackIds'][i]).split(';')
        temp2 = df2['Link'].tolist()
        temp1 =[]
        for j in range(0,len(temp4)):
            if temp4[j] in temp2:
                temp1.append(str(temp2.index(temp4[j])))
        temp3 = ';'.join(str(v) for v in temp1)
        df1.at[i,'TrackIds'] = temp3
       
df1.to_csv('./Crawl_data/playlist.csv',index = False)
df2.to_csv('./Crawl_data/track.csv',index = False)